In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re
from sklearn.model_selection import train_test_split # Pour split training/validation

In [2]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r', encoding='utf-8') as file: # Ajout de l'encodage
        text = file.read()
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]
    return sentences

file_path = 'output.txt'
text_data = file_to_sentence_list(file_path)

# 1. Tokenisation (peut toujours être fait une fois si le vocabulaire n'est pas trop grand)
tokenizer = Tokenizer(oov_token="<unk>") # Ajout d'un token pour les mots inconnus
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# 2. Déterminer max_sequence_len (peut être calculé sur un échantillon ou l'ensemble des données)
# Si le calcul complet de input_sequences fait crasher, vous devrez peut-être estimer ou fixer.
# Pour l'exemple, nous allons le faire une fois pour déterminer la longueur maximale.
# Si cette partie crashe aussi, vous devrez trouver une façon plus memory-friendly de le faire.
sequences_lengths = [len(tokenizer.texts_to_sequences([line])[0]) for line in text_data]
max_sequence_len = max(sequences_lengths) if sequences_lengths else 1

print(f"Total words in vocabulary: {total_words}")
print(f"Max sequence length: {max_sequence_len}")

# --- Stratégie avec un générateur pour l'entraînement ---
def data_generator(texts, tokenizer, max_len, total_words, batch_size):
    i = 0
    while True:
        batch_texts = texts[i:i + batch_size]
        if not batch_texts: # Recommence si on a atteint la fin de la liste
            i = 0
            batch_texts = texts[i:i + batch_size]

        batch_input_sequences = []
        for line in batch_texts:
            token_list = tokenizer.texts_to_sequences([line])[0]
            for j in range(1, len(token_list)):
                n_gram_sequence = token_list[:j+1]
                batch_input_sequences.append(n_gram_sequence)

        if not batch_input_sequences: # Gérer le cas où le batch est vide
            i += batch_size
            continue

        batch_input_sequences = np.array(pad_sequences(
            batch_input_sequences, maxlen=max_len, padding='pre'))

        X_batch, y_batch = batch_input_sequences[:, :-1], batch_input_sequences[:, -1]
        y_batch = tf.keras.utils.to_categorical(y_batch, num_classes=total_words)

        yield X_batch, y_batch
        i += batch_size

# Définir la taille du batch
batch_size = 512 # Commencez petit et augmentez si votre RAM le permet
# Divisez vos données en entraînement et validation (bonne pratique)
train_data, val_data = train_test_split(text_data, test_size=0.1, random_state=42)

# Créer les générateurs
train_generator = data_generator(train_data, tokenizer, max_sequence_len, total_words, batch_size)
val_generator = data_generator(val_data, tokenizer, max_sequence_len, total_words, batch_size)

# Calculer steps_per_epoch et validation_steps
# Ceci est une estimation, car chaque phrase génère plusieurs n-grams.
# Pour une estimation plus précise, vous devriez précalculer le nombre total de n-grams pour chaque split.
# Pour l'instant, une estimation basée sur le nombre de phrases suffit pour commencer.
num_train_n_grams_approx = sum(len(tokenizer.texts_to_sequences([line])[0]) - 1 for line in train_data if len(tokenizer.texts_to_sequences([line])[0]) > 1)
num_val_n_grams_approx = sum(len(tokenizer.texts_to_sequences([line])[0]) - 1 for line in val_data if len(tokenizer.texts_to_sequences([line])[0]) > 1)

steps_per_epoch = int(np.ceil(num_train_n_grams_approx / batch_size))
validation_steps = int(np.ceil(num_val_n_grams_approx / batch_size))

# Assurez-vous que steps_per_epoch et validation_steps ne sont pas nuls
if steps_per_epoch == 0:
    steps_per_epoch = 1
if validation_steps == 0:
    validation_steps = 1

print(f"Estimated training steps per epoch: {steps_per_epoch}")
print(f"Estimated validation steps: {validation_steps}")

Total words in vocabulary: 8568
Max sequence length: 89
Estimated training steps per epoch: 523
Estimated validation steps: 58


In [ ]:
# Définir le modèle (inchangé, mais notez l'input_length)
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entraîner le modèle avec le générateur
try:
    model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=1,
              verbose=1,
              validation_data=val_generator,
              validation_steps=validation_steps)
except Exception as e:
    print(f"An error occurred during training: {e}")
    print("Consider reducing batch_size, sequence_length, or embedding_dim.")

 36/523 ━━━━━━━━━━━━━━━━━━━━ 5:17 651ms/step - accuracy: 0.0380 - loss: 8.5706

KeyboardInterrupt: 

In [ ]:
# Entraîner le modèle avec le générateur
try:
    model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=1,
              verbose=1,
              validation_data=val_generator,
              validation_steps=validation_steps)
except Exception as e:
    print(f"An error occurred during training: {e}")
    print("Consider reducing batch_size, sequence_length, or embedding_dim.")

523/523 ━━━━━━━━━━━━━━━━━━━━ 361s 690ms/step - accuracy: 0.0946 - loss: 5.5438 - val_accuracy: 0.1387 - val_loss: 5.2387


In [ ]:
# Entraîner le modèle avec le générateur
try:
    model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=1,
              verbose=1,
              validation_data=val_generator,
              validation_steps=validation_steps)
except Exception as e:
    print(f"An error occurred during training: {e}")
    print("Consider reducing batch_size, sequence_length, or embedding_dim.")

523/523 ━━━━━━━━━━━━━━━━━━━━ 362s 692ms/step - accuracy: 0.1490 - loss: 5.0252 - val_accuracy: 0.1733 - val_loss: 4.9557


In [ ]:
import os # Assurez-vous que os est importé si ce n'est pas déjà fait

# --- Entraîner le modèle avec le générateur ---
try:
    print("Démarrage de l'entraînement du modèle...")
    model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=10, # J'ai laissé 10 époques comme dans votre dernier exemple
              verbose=1,
              validation_data=val_generator,
              validation_steps=validation_steps)
    print("\nEntraînement terminé avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant l'entraînement : {e}")
    print("Considérez de réduire davantage batch_size, la taille de l'unité LSTM, ou de plafonner max_sequence_len.")
    # On continue la sauvegarde même en cas d'erreur pour enregistrer l'état partiel du modèle.
    # Si le crash est critique (ex: OOM pendant model.fit lui-même), model.save() pourrait échouer aussi.

# --- Sauvegarde du modèle après l'entraînement (ou après un crash) ---
# Définissez le chemin local où vous voulez sauvegarder le modèle.
local_save_dir = './final_model_save/'
os.makedirs(local_save_dir, exist_ok=True) # Crée le dossier s'il n'existe pas

# Définissez le nom du fichier du modèle.
model_filename = 'final_trained_nlp_model.h5'
final_model_path = os.path.join(local_save_dir, model_filename)

try:
    # Sauvegarde du modèle complet (architecture, poids, état de l'optimiseur)
    model.save(final_model_path)
    print(f"\nModèle final sauvegardé avec succès dans : {final_model_path}")
except Exception as e:
    print(f"\nUne erreur s'est produite lors de la sauvegarde du modèle : {e}")
    print("La sauvegarde a échoué. Le modèle pourrait être incomplet ou l'environnement instable.")

# Vous pouvez ajouter le code pour charger et utiliser le modèle ici si vous le souhaitez,
# mais il est souvent plus clair de le faire dans une cellule séparée pour la prédiction.

# Exemple de code commenté pour charger le modèle :
# from tensorflow.keras.models import load_model
# try:
#     loaded_model = load_model(final_model_path)
#     print(f"Modèle chargé depuis {final_model_path} pour vérification/utilisation.")
#     # Vous pouvez maintenant utiliser 'loaded_model'
# except Exception as e:
#     print(f"Impossible de charger le modèle depuis {final_model_path}: {e}")

Démarrage de l'entraînement du modèle...
Epoch 1/10
523/523 ━━━━━━━━━━━━━━━━━━━━ 361s 690ms/step - accuracy: 0.1853 - loss: 4.7003 - val_accuracy: 0.1970 - val_loss: 4.7932
Epoch 2/10
523/523 ━━━━━━━━━━━━━━━━━━━━ 383s 732ms/step - accuracy: 0.2079 - loss: 4.4751 - val_accuracy: 0.2126 - val_loss: 4.6876
Epoch 3/10
523/523 ━━━━━━━━━━━━━━━━━━━━ 361s 690ms/step - accuracy: 0.2260 - loss: 4.3007 - val_accuracy: 0.2248 - val_loss: 4.6104
Epoch 4/10
523/523 ━━━━━━━━━━━━━━━━━━━━ 361s 690ms/step - accuracy: 0.2405 - loss: 4.1549 - val_accuracy: 0.2347 - val_loss: 4.5560
Epoch 5/10
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 630ms/step - accuracy: 0.2523 - loss: 4.0248

In [ ]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Next predicted words: Pizza have different  become a symbol of comfort


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model # <-- Importez load_model

# --- 1. Définir le chemin vers votre modèle sauvegardé localement ---
# Assurez-vous que ce chemin correspond à l'endroit où vous avez sauvegardé votre modèle.
local_save_dir = './final_model_save/'
model_filename = 'final_trained_nlp_model.h5'
final_model_path = os.path.join(local_save_dir, model_filename)

# --- 2. Charger le modèle ---
print(f"Tentative de chargement du modèle depuis : {final_model_path}")
try:
    loaded_model = load_model(final_model_path)
    print("Modèle chargé avec succès pour reprendre l'entraînement !")
    # Affichez un résumé pour vérifier que le modèle est bien chargé
    loaded_model.summary()
except Exception as e:
    print(f"Erreur lors du chargement du modèle depuis {final_model_path} : {e}")
    print("Veuillez vérifier le chemin du fichier et si le fichier .h5 existe.")
    # Sortez ou arrêtez l'exécution si le modèle ne peut pas être chargé
    exit()

# --- 3. Reprendre l'entraînement ---
# Vous avez déjà défini les générateurs (train_generator, val_generator),
# les steps_per_epoch, validation_steps, et d'autres paramètres dans les cellules précédentes.
# Assurez-vous qu'ils sont toujours disponibles et correctement configurés dans votre environnement.

# Vous pouvez également reconfigurer des callbacks si vous le souhaitez pour cette nouvelle phase d'entraînement.
# Par exemple, pour continuer à sauvegarder le MEILLEUR modèle sur Drive, ou un nouveau callback de checkpoint local.

# Optionnel: Définir un nouveau chemin de sauvegarde pour cette phase d'entraînement,
# si vous ne voulez pas écraser l'ancien "final_trained_nlp_model.h5" immédiatement.
# Ou utilisez un ModelCheckpoint pour sauvegarder à chaque époque comme précédemment.
#
# Exemple de ModelCheckpoint pour cette nouvelle phase (localement):
# from tensorflow.keras.callbacks import ModelCheckpoint
# continue_save_dir = './continued_training_saves/'
# os.makedirs(continue_save_dir, exist_ok=True)
# continue_filepath = os.path.join(continue_save_dir, "continued_model-epoch-{epoch:02d}-val_loss-{val_loss:.4f}.h5")
#
# checkpoint_continue = ModelCheckpoint(
#     continue_filepath,
#     monitor='val_loss',
#     verbose=1,
#     save_best_only=False, # Sauvegarde chaque époque pour ne rien perdre
#     mode='min',
#     save_freq='epoch'
# )
# callbacks_for_continue = [checkpoint_continue]


print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # Entraînez pour quelques époques supplémentaires (ajustez ce nombre)
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Activez si vous utilisez le ModelCheckpoint ci-dessus
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Si vous n'avez pas utilisé ModelCheckpoint pour cette phase, sauvegardez le modèle final manuellement.
new_final_model_path = os.path.join(local_save_dir, 'final_trained_nlp_model_continued.h5')
try:
    loaded_model.save(new_final_model_path)
    print(f"\nModèle entraîné en continu sauvegardé dans : {new_final_model_path}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")

Tentative de chargement du modèle depuis : ./final_model_save/final_trained_nlp_model.h5


Modèle chargé avec succès pour reprendre l'entraînement !


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 88, 10)         │        85,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        71,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8568)           │     1,105,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,262,122 (4.81 MB)

 Trainable params: 1,262,120 (4.81 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


Démarrage de la reprise de l'entraînement...
Epoch 1/5



Une erreur s'est produite pendant la reprise de l'entraînement : numpy() is only available when eager execution is enabled.
Veuillez vérifier les générateurs de données et les ressources disponibles.

Modèle entraîné en continu sauvegardé dans : ./final_model_save/final_trained_nlp_model_continued.h5


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
# Importez pad_sequences de keras.utils si celle de preprocessing.sequence pose problème ou n'est pas présente.
# Sinon, tf.keras.preprocessing.sequence.pad_sequences est l'approche standard.
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import re

# --- Fonctions et configurations préalables ---

def file_to_sentence_list(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]
    return sentences

# Charger les données (assurez-vous que 'output.txt' existe)
file_path = 'output.txt'
text_data = file_to_sentence_list(file_path)

# 1. Tokenisation
tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# 2. Déterminer max_sequence_len
sequences_lengths = [len(tokenizer.texts_to_sequences([line])[0]) for line in text_data]
max_sequence_len = max(sequences_lengths) if sequences_lengths else 1

print(f"Total words in vocabulary: {total_words}")
print(f"Max sequence length: {max_sequence_len}")

# --- Stratégie avec un générateur pour l'entraînement ---
def data_generator(texts, tokenizer, max_len, total_words, batch_size):
    i = 0
    while True:
        if i == 0:
            np.random.shuffle(texts)

        batch_texts = texts[i:i + batch_size]
        if not batch_texts:
            i = 0
            np.random.shuffle(texts)
            batch_texts = texts[i:i + batch_size]

        batch_input_sequences_temp = []
        for line in batch_texts:
            token_list = tokenizer.texts_to_sequences([line])[0]
            if len(token_list) > 1:
                for j in range(1, len(token_list)):
                    n_gram_sequence = token_list[:j+1]
                    batch_input_sequences_temp.append(n_gram_sequence)

        if not batch_input_sequences_temp:
            i += batch_size
            continue

        # Convertir en tableau NumPy D'ABORD, puis effectuer les opérations
        padded_sequences_np = np.array(pad_sequences(batch_input_sequences_temp, maxlen=max_len, padding='pre'))

        X_batch_np = padded_sequences_np[:, :-1]
        y_batch_np = padded_sequences_np[:, -1]

        # S'assurer que to_categorical reçoit un tableau NumPy
        y_batch_one_hot_np = tf.keras.utils.to_categorical(y_batch_np, num_classes=total_words)

        # Enfin, convertir les tableaux NumPy en tenseurs TensorFlow pour la sortie du générateur
        X_batch = tf.constant(X_batch_np, dtype=tf.int32)
        y_batch = tf.constant(y_batch_one_hot_np, dtype=tf.float32) # Les encodings one-hot sont des floats

        yield X_batch, y_batch
        i += batch_size

# Définir la taille du batch
batch_size = 512

# Divisez vos données en entraînement et validation
train_data, val_data = train_test_split(text_data, test_size=0.1, random_state=42)

# Créer les générateurs
train_generator = data_generator(train_data, tokenizer, max_sequence_len, total_words, batch_size)
val_generator = data_generator(val_data, tokenizer, max_sequence_len, total_words, batch_size)

def count_n_grams(texts, tokenizer):
    count = 0
    for line in texts:
        token_list = tokenizer.texts_to_sequences([line])[0]
        if len(token_list) > 1:
            count += (len(token_list) - 1)
    return count

num_train_n_grams_approx = count_n_grams(train_data, tokenizer)
num_val_n_grams_approx = count_n_grams(val_data, tokenizer)

steps_per_epoch = int(np.ceil(num_train_n_grams_approx / batch_size))
validation_steps = int(np.ceil(num_val_n_grams_approx / batch_size))

if steps_per_epoch == 0:
    steps_per_epoch = 1
if validation_steps == 0:
    validation_steps = 1

print(f"Estimated total n-grams for training: {num_train_n_grams_approx}")
print(f"Estimated training steps per epoch: {steps_per_epoch}")
print(f"Estimated total n-grams for validation: {num_val_n_grams_approx}")
print(f"Estimated validation steps: {validation_steps}")

# --- Section de chargement et entraînement du modèle ---
local_save_dir = './final_model_save/'

model_filename_keras = 'final_trained_nlp_model.keras'
final_model_path_keras = os.path.join(local_save_dir, model_filename_keras)

os.makedirs(local_save_dir, exist_ok=True)

print(f"\nTentative de chargement du modèle depuis : {final_model_path_keras}")
loaded_model = None
try:
    loaded_model = load_model(final_model_path_keras)
    print("Modèle chargé avec succès pour reprendre l'entraînement !")
    loaded_model.summary()
except Exception as e:
    print(f"Erreur lors du chargement du modèle depuis {final_model_path_keras} : {e}")
    print("Veuillez vérifier le chemin du fichier et si le fichier .keras existe.")

    model_filename_h5 = 'final_trained_nlp_model.h5'
    final_model_path_h5 = os.path.join(local_save_dir, model_filename_h5)
    print(f"\nTentative de chargement de l'ancien format HDF5 ({final_model_path_h5})...")
    try:
        loaded_model = load_model(final_model_path_h5)
        print("Modèle HDF5 chargé avec succès. Il sera sauvegardé au nouveau format après l'entraînement.")
        loaded_model.summary()
    except Exception as e_h5:
        print(f"Erreur lors du chargement de l'ancien format HDF5 : {e_h5}")
        print("\nRecréation du modèle car le chargement a échoué (ni .keras ni .h5 trouvés ou valides)...")
        embedding_dim = 100
        model = Sequential()
        model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_len - 1))
        model.add(LSTM(150, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(100))
        model.add(Dense(total_words, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        loaded_model = model
        loaded_model.summary()


print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5,
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

try:
    if loaded_model:
        loaded_model.save(final_model_path_keras)
        print(f"\nModèle entraîné en continu sauvegardé au format .keras dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")

Total words in vocabulary: 8568
Max sequence length: 89
Estimated total n-grams for training: 267583
Estimated training steps per epoch: 523
Estimated total n-grams for validation: 29663
Estimated validation steps: 58

Tentative de chargement du modèle depuis : ./final_model_save/final_trained_nlp_model.keras
Modèle chargé avec succès pour reprendre l'entraînement !


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 88, 10)         │        85,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        71,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8568)           │     1,105,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,786,362 (14.44 MB)

 Trainable params: 1,262,120 (4.81 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,524,242 (9.63 MB)


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 559s 1s/step - accuracy: 0.1859 - loss: 4.6914 - val_accuracy: 0.1974 - val_loss: 4.8164
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 557s 1s/step - accuracy: 0.2087 - loss: 4.4383 - val_accuracy: 0.2142 - val_loss: 4.7287
Epoch 3/5
 44/523 ━━━━━━━━━━━━━━━━━━━━ 7:30 941ms/step - accuracy: 0.2241 - loss: 4.2870

KeyboardInterrupt: 

In [ ]:
try:
    loaded_model.save(final_model_path_keras)
    print(f"\nModèle entraîné en continu sauvegardé au format .keras dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Modèle entraîné en continu sauvegardé au format .keras dans : ./final_model_save/final_trained_nlp_model.keras


In [ ]:
# Assurez-vous d'avoir le modèle chargé (loaded_model) et le tokenizer disponibles
# Utilisez loaded_model pour la prédiction

# Generate next word predictions
seed_text = "hi " # Texte d'amorçage
next_words = 2 # Nombre de mots à prédire

# Utilisez le modèle qui a été chargé ou récréé et entraîné
model_to_predict = loaded_model

print(f"Seed text: {seed_text}")
print(f"Predicting {next_words} next words...")

for _ in range(next_words):
    # 1. Tokenisation du texte d'amorçage
    # Convertit le texte en une séquence d'entiers selon le vocabulaire du tokenizer
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # 2. Padding de la séquence
    # Le modèle attend une séquence d'une longueur fixe (max_sequence_len-1).
    # 'pre' padding ajoute des zéros au début si la séquence est trop courte.
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len - 1, padding='pre')

    # 3. Prédiction du mot suivant
    # model.predict() renvoie les probabilités pour chaque mot du vocabulaire.
    # On utilise expand_dims si le modèle attend une dimension de batch explicite (par exemple (1, seq_len))
    # Cependant, pad_sequences avec [token_list] renvoie déjà (1, seq_len)
    predicted_probs = model_to_predict.predict(token_list, verbose=0) # verbose=0 pour ne pas spammer la console

    # 4. Obtention du mot le plus probable
    # np.argmax() trouve l'indice (ID du mot) de la probabilité la plus élevée.
    # tokenizer.index_word convertit cet ID en mot.
    predicted_word_index = np.argmax(predicted_probs)
    predicted_word = tokenizer.index_word[predicted_word_index]

    # 5. Ajout du mot prédit au texte d'amorçage pour la prochaine itération
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

Seed text: hi 
Predicting 2 next words...
Next predicted words: hi  is a


In [3]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model # Nécessaire pour charger

# Définissez le chemin où votre modèle est sauvegardé
local_save_dir = './final_model_save/'
model_filename_keras = 'final_trained_nlp_model.keras' # Assurez-vous d'utiliser le bon format/nom
final_model_path_keras = os.path.join(local_save_dir, model_filename_keras)

loaded_model = None
try:
    print(f"Tentative de chargement du modèle depuis : {final_model_path_keras}")
    loaded_model = load_model(final_model_path_keras)
    print("Modèle chargé avec succès pour reprendre l'entraînement !")
    loaded_model.summary() # Vérifiez que le modèle est bien chargé et a la bonne structure
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")
    print("Impossible de charger le modèle. Vérifiez le chemin ou créez un nouveau modèle.")
    # Ici, vous devriez soit sortir (exit()), soit recréer un nouveau modèle
    # si vous voulez commencer un entraînement from scratch si aucun modèle n'est trouvé.
    # Pour la continuation, il est essentiel qu'un modèle soit chargé.
    # (Le code que vous avez déjà inclut la recréation si le chargement échoue)
    exit() # Arrête le script si le modèle ne peut pas être chargé

Tentative de chargement du modèle depuis : ./final_model_save/final_trained_nlp_model.keras
Modèle chargé avec succès pour reprendre l'entraînement !


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 88, 10)         │        85,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        71,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8568)           │     1,105,272 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,786,362 (14.44 MB)

 Trainable params: 1,262,120 (4.81 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,524,242 (9.63 MB)

In [ ]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 559s 1s/step - accuracy: 0.2295 - loss: 4.2322 - val_accuracy: 0.2301 - val_loss: 4.6463
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 563s 1s/step - accuracy: 0.2453 - loss: 4.0673 - val_accuracy: 0.2397 - val_loss: 4.5999
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 564s 1s/step - accuracy: 0.2580 - loss: 3.9320 - val_accuracy: 0.2459 - val_loss: 4.5795
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 567s 1s/step - accuracy: 0.2674 - loss: 3.8271 - val_accuracy: 0.2492 - val_loss: 4.5777
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 564s 1s/step - accuracy: 0.2753 - loss: 3.7410 - val_accuracy: 0.2513 - val_loss: 4.5785

Reprise de l'entraînement terminée avec succès.

Modèle entraîné en continu sauvegardé dans : ./final_model_save/final_trained_nlp_model.keras


In [ ]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 565s 1s/step - accuracy: 0.2829 - loss: 3.6643 - val_accuracy: 0.2552 - val_loss: 4.5868
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 563s 1s/step - accuracy: 0.2902 - loss: 3.5998 - val_accuracy: 0.2574 - val_loss: 4.5991
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 561s 1s/step - accuracy: 0.2960 - loss: 3.5426 - val_accuracy: 0.2586 - val_loss: 4.6192
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 565s 1s/step - accuracy: 0.3014 - loss: 3.4931 - val_accuracy: 0.2586 - val_loss: 4.6347
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 567s 1s/step - accuracy: 0.3063 - loss: 3.4467 - val_accuracy: 0.2589 - val_loss: 4.6547

Reprise de l'entraînement terminée avec succès.

Modèle entraîné en continu sauvegardé dans : ./final_model_save/final_trained_nlp_model.keras


In [ ]:
# Assurez-vous d'avoir le modèle chargé (loaded_model) et le tokenizer disponibles
# Utilisez loaded_model pour la prédiction

# Generate next word predictions
seed_text = "I love playing with him because he is" # Texte d'amorçage
next_words = 10 # Nombre de mots à prédire

# Utilisez le modèle qui a été chargé ou récréé et entraîné
model_to_predict = loaded_model

print(f"Seed text: {seed_text}")
print(f"Predicting {next_words} next words...")

for _ in range(next_words):
    # 1. Tokenisation du texte d'amorçage
    # Convertit le texte en une séquence d'entiers selon le vocabulaire du tokenizer
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # 2. Padding de la séquence
    # Le modèle attend une séquence d'une longueur fixe (max_sequence_len-1).
    # 'pre' padding ajoute des zéros au début si la séquence est trop courte.
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len - 1, padding='pre')

    # 3. Prédiction du mot suivant
    # model.predict() renvoie les probabilités pour chaque mot du vocabulaire.
    # On utilise expand_dims si le modèle attend une dimension de batch explicite (par exemple (1, seq_len))
    # Cependant, pad_sequences avec [token_list] renvoie déjà (1, seq_len)
    predicted_probs = model_to_predict.predict(token_list, verbose=0) # verbose=0 pour ne pas spammer la console

    # 4. Obtention du mot le plus probable
    # np.argmax() trouve l'indice (ID du mot) de la probabilité la plus élevée.
    # tokenizer.index_word convertit cet ID en mot.
    predicted_word_index = np.argmax(predicted_probs)
    predicted_word = tokenizer.index_word[predicted_word_index]

    # 5. Ajout du mot prédit au texte d'amorçage pour la prochaine itération
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

Seed text: I love playing with him because he is
Predicting 10 next words...
Next predicted words: I love playing with him because he is a very good snowboarder and i have a very disappointing


In [ ]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 554s 1s/step - accuracy: 0.3110 - loss: 3.4051 - val_accuracy: 0.2597 - val_loss: 4.6792
Epoch 2/5


In [4]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 366s 691ms/step - accuracy: 0.3108 - loss: 3.4060 - val_accuracy: 0.2588 - val_loss: 4.6781
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 359s 686ms/step - accuracy: 0.3144 - loss: 3.3718 - val_accuracy: 0.2600 - val_loss: 4.7025
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 361s 691ms/step - accuracy: 0.3186 - loss: 3.3369 - val_accuracy: 0.2603 - val_loss: 4.7313
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 359s 687ms/step - accuracy: 0.3221 - loss: 3.3091 - val_accuracy: 0.2605 - val_loss: 4.7496
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 383s 732ms/step - accuracy: 0.3260 - loss: 3.2764 - val_accuracy: 0.2608 - val_loss: 4.7729

Reprise de l'entraînement terminée avec succès.

Modèle entraîné en continu sauvegardé dans : ./final_model_save/final_trained_nlp_model.keras


In [9]:
# Assurez-vous d'avoir le modèle chargé (loaded_model) et le tokenizer disponibles
# Utilisez loaded_model pour la prédiction

# Generate next word predictions
seed_text = "I was sad last day in the festival i felt" # Texte d'amorçage
next_words = 6 # Nombre de mots à prédire

# Utilisez le modèle qui a été chargé ou récréé et entraîné
model_to_predict = loaded_model

print(f"Seed text: {seed_text}")
print(f"Predicting {next_words} next words...")

for _ in range(next_words):
    # 1. Tokenisation du texte d'amorçage
    # Convertit le texte en une séquence d'entiers selon le vocabulaire du tokenizer
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # 2. Padding de la séquence
    # Le modèle attend une séquence d'une longueur fixe (max_sequence_len-1).
    # 'pre' padding ajoute des zéros au début si la séquence est trop courte.
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len - 1, padding='pre')

    # 3. Prédiction du mot suivant
    # model.predict() renvoie les probabilités pour chaque mot du vocabulaire.
    # On utilise expand_dims si le modèle attend une dimension de batch explicite (par exemple (1, seq_len))
    # Cependant, pad_sequences avec [token_list] renvoie déjà (1, seq_len)
    predicted_probs = model_to_predict.predict(token_list, verbose=0) # verbose=0 pour ne pas spammer la console

    # 4. Obtention du mot le plus probable
    # np.argmax() trouve l'indice (ID du mot) de la probabilité la plus élevée.
    # tokenizer.index_word convertit cet ID en mot.
    predicted_word_index = np.argmax(predicted_probs)
    predicted_word = tokenizer.index_word[predicted_word_index]

    # 5. Ajout du mot prédit au texte d'amorçage pour la prochaine itération
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

Seed text: I was sad last day in the festival i felt
Predicting 6 next words...
Next predicted words: I was sad last day in the festival i felt very disappointed with the musical show


In [10]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 363s 693ms/step - accuracy: 0.3293 - loss: 3.2506 - val_accuracy: 0.2612 - val_loss: 4.8004
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 360s 687ms/step - accuracy: 0.3327 - loss: 3.2247 - val_accuracy: 0.2606 - val_loss: 4.8223
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 358s 684ms/step - accuracy: 0.3354 - loss: 3.2034 - val_accuracy: 0.2612 - val_loss: 4.8479
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 383s 732ms/step - accuracy: 0.3370 - loss: 3.1880 - val_accuracy: 0.2609 - val_loss: 4.8722
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 382s 730ms/step - accuracy: 0.3409 - loss: 3.1599 - val_accuracy: 0.2600 - val_loss: 4.9035

Reprise de l'entraînement terminée avec succès.

Modèle entraîné en continu sauvegardé dans : ./final_model_save/final_trained_nlp_model.keras


In [11]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 363s 694ms/step - accuracy: 0.3428 - loss: 3.1426 - val_accuracy: 0.2605 - val_loss: 4.9204
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 363s 694ms/step - accuracy: 0.3456 - loss: 3.1219 - val_accuracy: 0.2597 - val_loss: 4.9451
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 383s 732ms/step - accuracy: 0.3477 - loss: 3.1053 - val_accuracy: 0.2614 - val_loss: 4.9639
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 382s 731ms/step - accuracy: 0.3504 - loss: 3.0866 - val_accuracy: 0.2578 - val_loss: 4.9911
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 382s 731ms/step - accuracy: 0.3515 - loss: 3.0772 - val_accuracy: 0.2585 - val_loss: 5.0108

Reprise de l'entraînement terminée avec succès.

Modèle entraîné en continu sauvegardé dans : ./final_model_save/final_trained_nlp_model.keras


In [ ]:
print("\nDémarrage de la reprise de l'entraînement...")
try:
    loaded_model.fit(train_generator,
                     steps_per_epoch=steps_per_epoch,
                     epochs=5, # C'est le nombre d'époques SUPPLÉMENTAIRES que vous voulez entraîner
                               # Si vous avez déjà fait 10 époques et que vous en ajoutez 5 ici,
                               # le modèle aura été entraîné pendant 15 époques au total.
                     verbose=1,
                     validation_data=val_generator,
                     validation_steps=validation_steps,
                     # callbacks=callbacks_for_continue # Si vous avez des callbacks supplémentaires pour cette phase
                    )
    print("\nReprise de l'entraînement terminée avec succès.")
except Exception as e:
    print(f"\nUne erreur s'est produite pendant la reprise de l'entraînement : {e}")
    print("Veuillez vérifier les générateurs de données et les ressources disponibles.")

# --- Optionnel : Sauvegarder le modèle après cette phase d'entraînement continu ---
# Il est toujours bon de sauvegarder le modèle après chaque phase d'entraînement.
try:
    if loaded_model:
        loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
        print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde du modèle entraîné en continu : {e}")


Démarrage de la reprise de l'entraînement...
Epoch 1/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 363s 694ms/step - accuracy: 0.3547 - loss: 3.0548 - val_accuracy: 0.2577 - val_loss: 5.0316
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 383s 732ms/step - accuracy: 0.3565 - loss: 3.0433 - val_accuracy: 0.2567 - val_loss: 5.0573
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 382s 731ms/step - accuracy: 0.3571 - loss: 3.0345 - val_accuracy: 0.2586 - val_loss: 5.0783
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 382s 730ms/step - accuracy: 0.3599 - loss: 3.0153 - val_accuracy: 0.2582 - val_loss: 5.1061
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 382s 731ms/step - accuracy: 0.3609 - loss: 3.0070 - val_accuracy: 0.2583 - val_loss: 5.1167


In [2]:
local_save_dir = './final_model_save/'
model_filename_keras = 'final_trained_nlp_model.keras' # Assurez-vous d'utiliser le bon format/nom
final_model_path_keras = os.path.join(local_save_dir, model_filename_keras)
loaded_model.save(final_model_path_keras) # Sauvegarde à nouveau au même emplacement (ou un nouveau)
print(f"\nModèle entraîné en continu sauvegardé dans : {final_model_path_keras}")


IndentationError: unexpected indent (ipython-input-3992040528.py, line 4)